<a href="https://colab.research.google.com/github/PujithaReddyKethireddy/YOLO_MODEL/blob/main/yolo_bas1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Sep  7 13:55:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!pip install torch torchvision ultralytics
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r /content/yolov5/requirements.txt

Mounted at /content/gdrive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.1 MB/s eta 0:00:00
Cloning into 'yolov5'...
remote: Enumerating objects: 17564, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 17564 (delta 33), reused 7 (delta 7), pack-reused 17510 (from 3)
Receiving objects: 100% (17564/17564), 16.69 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (12031/12031), done.


In [3]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path

def convert_xml_to_yolo(xml_files, output_dir, class_names):
    os.makedirs(output_dir, exist_ok=True)
    for xml_file in xml_files:
        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            size = root.find('size')
            if size is None:
                print(f"⚠️ No <size> tag in {xml_file}")
                continue
            width = int(size.find('width').text)
            height = int(size.find('height').text)
            if width == 0 or height == 0:
                print(f"⚠️ Invalid size in {xml_file}")
                continue
            yolo_lines = []
            for obj in root.findall('object'):
                class_name = obj.find('name').text.lower()
                if class_name not in class_names:
                    continue
                class_id = class_names.index(class_name)
                bbox = obj.find('bndbox')
                if bbox is None:
                    continue
                xmin = float(bbox.find('xmin').text)
                ymin = float(bbox.find('ymin').text)
                xmax = float(bbox.find('xmax').text)
                ymax = float(bbox.find('ymax').text)
                x_center = (xmin + xmax) / 2 / width
                y_center = (ymin + ymax) / 2 / height
                bb_width = (xmax - xmin) / width
                bb_height = (ymax - ymin) / height
                yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {bb_width:.6f} {bb_height:.6f}")
            if yolo_lines:
                output_file = os.path.join(output_dir, os.path.splitext(os.path.basename(xml_file))[0] + '.txt')
                with open(output_file, 'w') as f:
                    f.write('\n'.join(yolo_lines))
        except Exception as e:
            print(f"⚠️ Error processing {xml_file}: {e}")

class_names = ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']
annotation_files = []
for root, _, files in os.walk('/content/gdrive/MyDrive/PCB_DATASET/Annotations'):
    annotation_files.extend([os.path.join(root, f) for f in files if f.endswith('.xml')])
convert_xml_to_yolo(annotation_files, '/content/PCB_DATASET/yolo_annotations', class_names)

# Verify conversion
!ls /content/PCB_DATASET/yolo_annotations | wc -l
!cat /content/PCB_DATASET/yolo_annotations/01_spur_17.txt

120
4 0.732037 0.388714 0.010547 0.027112
4 0.640574 0.482346 0.013514 0.018916
4 0.825148 0.386822 0.014173 0.020807

In [4]:
import os
from pathlib import Path

def analyze_dataset_structure():
    base_path = '/content/gdrive/MyDrive/PCB_DATASET/images'
    all_images = []
    for root, _, files in os.walk(base_path):
        image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
        if image_files:
            all_images.extend([os.path.join(root, f) for f in image_files])
            print(f"📁 {root}: {len(image_files)} images")
    annotation_files = [os.path.join('/content/PCB_DATASET/yolo_annotations', f) for f in os.listdir('/content/PCB_DATASET/yolo_annotations') if f.endswith('.txt')]
    print(f"📄 /content/PCB_DATASET/yolo_annotations: {len(annotation_files)} .txt files")
    print(f"📊 Total images: {len(all_images)}, Total annotations: {len(annotation_files)}")
    return all_images, annotation_files

all_images, annotation_files = analyze_dataset_structure()

📁 /content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper: 20 images
📁 /content/gdrive/MyDrive/PCB_DATASET/images/Short: 20 images
📁 /content/gdrive/MyDrive/PCB_DATASET/images/Spur: 20 images
📁 /content/gdrive/MyDrive/PCB_DATASET/images/Missing_hole: 20 images
📁 /content/gdrive/MyDrive/PCB_DATASET/images/Open_circuit: 20 images
📁 /content/gdrive/MyDrive/PCB_DATASET/images/Mouse_bite: 20 images
📄 /content/PCB_DATASET/yolo_annotations: 120 .txt files
📊 Total images: 120, Total annotations: 120


In [5]:
def match_images_with_annotations(all_images, annotation_files):
    matched_pairs = []
    image_lookup = {os.path.splitext(os.path.basename(img))[0]: img for img in all_images}
    annotation_lookup = {os.path.splitext(os.path.basename(ann))[0]: ann for ann in annotation_files}
    for img_name, img_path in image_lookup.items():
        if img_name in annotation_lookup:
            matched_pairs.append((img_path, annotation_lookup[img_name]))
    unmatched_images = [img for img in all_images if os.path.splitext(os.path.basename(img))[0] not in annotation_lookup]
    unmatched_annotations = [ann for ann in annotation_files if os.path.splitext(os.path.basename(ann))[0] not in image_lookup]
    print(f"✅ Matched pairs: {len(matched_pairs)}")
    print(f"⚠️ Unmatched images: {len(unmatched_images)}")
    print(f"⚠️ Unmatched annotations: {len(unmatched_annotations)}")
    if unmatched_images or unmatched_annotations:
        print("Sample unmatched images:", [os.path.basename(img) for img in unmatched_images[:3]])
        print("Sample unmatched annotations:", [os.path.basename(ann) for ann in unmatched_annotations[:3]])
    return matched_pairs

matched_pairs = match_images_with_annotations(all_images, annotation_files)

✅ Matched pairs: 120
⚠️ Unmatched images: 0
⚠️ Unmatched annotations: 0


In [6]:
import shutil
import random
from pathlib import Path

def prepare_yolo_dataset(matched_pairs):
    output_dir = '/content/PCB_DATASET/yolo_final_dataset'
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)
    random.seed(42)
    random.shuffle(matched_pairs)
    n_total = len(matched_pairs)
    n_train = int(0.8 * n_total)  # ~554
    n_val = int(0.15 * n_total)   # ~103
    n_test = n_total - n_train - n_val  # ~36
    splits = {
        'train': matched_pairs[:n_train],
        'val': matched_pairs[n_train:n_train + n_val],
        'test': matched_pairs[n_train + n_val:]
    }
    for split_name, pairs in splits.items():
        for i, (img_path, ann_path) in enumerate(pairs):
            base_name = f"{split_name}_{i:04d}_{os.path.basename(img_path)}"
            shutil.copy2(img_path, os.path.join(output_dir, 'images', split_name, base_name))
            label_name = os.path.splitext(base_name)[0] + '.txt'
            shutil.copy2(ann_path, os.path.join(output_dir, 'labels', split_name, label_name))
        print(f"📁 {split_name}: {len(pairs)} pairs")
    return output_dir

output_dir = prepare_yolo_dataset(matched_pairs)

📁 train: 96 pairs
📁 val: 18 pairs
📁 test: 6 pairs


In [7]:
import yaml

def create_dataset_config(output_dir):
    config = {
        'path': output_dir,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': 6,
        'names': ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']
    }
    config_path = os.path.join(output_dir, 'dataset.yaml')
    with open(config_path, 'w') as f:
        yaml.dump(config, f)

    # No augmentation
    hyp = {
        'lr0': 0.01,
        'lrf': 0.1,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3.0,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'box': 0.05,
        'cls': 0.5,
        'cls_pw': 1.0,
        'obj': 1.0,
        'obj_pw': 1.0,
        'iou_t': 0.2,
        'anchor_t': 4.0,
        'fl_gamma': 0.0,
        'hsv_h': 0.0,    # No hue
        'hsv_s': 0.0,    # No saturation
        'hsv_v': 0.0,    # No value
        'degrees': 0.0,  # No rotation
        'translate': 0.0,# No translation
        'scale': 0.0,    # No scaling
        'shear': 0.0,
        'perspective': 0.0,
        'flipud': 0.0,   # No vertical flip
        'fliplr': 0.0,   # No horizontal flip
        'mosaic': 0.0,   # No mosaic
        'mixup': 0.0,
        'copy_paste': 0.0
    }
    hyp_path = os.path.join(output_dir, 'hyp.no_augmentation.yaml')
    with open(hyp_path, 'w') as f:
        yaml.dump(hyp, f)
    return config_path, hyp_path

output_dir = '/content/PCB_DATASET/yolo_final_dataset'
config_path, hyp_path = create_dataset_config(output_dir)

In [8]:
import subprocess
import os

def train_yolo_model(config_path, hyp_path):
    os.chdir('/content/yolov5')
    train_cmd = [
        'python', 'train.py',
        '--data', config_path,
        '--hyp', hyp_path,
        '--weights', 'yolov5s.pt',
        '--epochs', '100',
        '--batch-size', '16',
        '--img', '640',
        '--project', 'pcb_defect_detection',
        '--name', 'final_model',
        '--device', '0'
    ]
    subprocess.run(train_cmd, check=True)
    return "/content/yolov5/pcb_defect_detection/final_model"

config_path = '/content/PCB_DATASET/yolo_final_dataset/dataset.yaml'
hyp_path = '/content/PCB_DATASET/yolo_final_dataset/hyp.no_augmentation.yaml'
results_dir = train_yolo_model(config_path, hyp_path)

In [9]:
results_dir = '/content/yolov5/pcb_defect_detection/final_model'
!cp {results_dir}/weights/best.pt /content/gdrive/MyDrive/YOLOv5_model.pt
!zip -r /content/results.zip {results_dir}
!cp /content/results.zip /content/gdrive/MyDrive/results.zip

  adding: content/yolov5/pcb_defect_detection/final_model/ (stored 0%)
  adding: content/yolov5/pcb_defect_detection/final_model/opt.yaml (deflated 50%)
  adding: content/yolov5/pcb_defect_detection/final_model/val_batch0_pred.jpg (deflated 18%)
  adding: content/yolov5/pcb_defect_detection/final_model/hyp.yaml (deflated 46%)
  adding: content/yolov5/pcb_defect_detection/final_model/P_curve.png (deflated 9%)
  adding: content/yolov5/pcb_defect_detection/final_model/F1_curve.png (deflated 7%)
  adding: content/yolov5/pcb_defect_detection/final_model/labels.jpg (deflated 37%)
  adding: content/yolov5/pcb_defect_detection/final_model/confusion_matrix.png (deflated 24%)
  adding: content/yolov5/pcb_defect_detection/final_model/train_batch2.jpg (deflated 23%)
  adding: content/yolov5/pcb_defect_detection/final_model/train_batch1.jpg (deflated 23%)
  adding: content/yolov5/pcb_defect_detection/final_model/labels_correlogram.jpg (deflated 43%)
  adding: content/yolov5/pcb_defect_detection/fin

In [10]:
!python /content/yolov5/detect.py --weights /content/gdrive/MyDrive/YOLOv5_model.pt --source /content/PCB_DATASET/yolo_final_dataset/images/test --project /content/results --name test_inference
!cp -r /content/results/test_inference /content/gdrive/MyDrive/results/test_inference
!python /content/yolov5/detect.py --weights /content/gdrive/MyDrive/YOLOv5_model.pt --source /content/gdrive/MyDrive/PCB_DATASET/PCB_USED --project /content/results --name pcb_used_inference
!cp -r /content/results/pcb_used_inference /content/gdrive/MyDrive/results/pcb_used_inference

detect: weights=['/content/gdrive/MyDrive/YOLOv5_model.pt'], source=/content/PCB_DATASET/yolo_final_dataset/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/results, name=test_inference, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-430-g459d8bf0 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
image 1/6 /content/PCB_DATASET/yolo_final_dataset/images/test/test_0000_01_short_17.jpg: 352x640 4 shorts, 33.2ms
image 2/6 /content/PCB_DATASET/yolo_final_dataset/images/test/test_0001_01_short_12.jpg: 352x640 5 shorts, 7.5ms
image 3/6 /content/PCB_DATASET

In [11]:
!zip -r /content/PCB_DATASET.zip /content/gdrive/MyDrive/PCB_DATASET/images /content/gdrive/MyDrive/PCB_DATASET/Annotations
!cp /content/PCB_DATASET.zip /content/gdrive/MyDrive/PCB_DATASET.zip

  adding: content/gdrive/MyDrive/PCB_DATASET/images/ (stored 0%)
  adding: content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper/ (stored 0%)
  adding: content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper/01_spurious_copper_12.jpg (deflated 1%)
  adding: content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper/01_spurious_copper_10.jpg (deflated 1%)
  adding: content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper/01_spurious_copper_13.jpg (deflated 1%)
  adding: content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper/01_spurious_copper_04.jpg (deflated 1%)
  adding: content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper/01_spurious_copper_08.jpg (deflated 1%)
  adding: content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper/01_spurious_copper_06.jpg (deflated 1%)
  adding: content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper/01_spurious_copper_09.jpg (deflated 1%)
  adding: content/gdrive/MyDrive/PCB_DATASET/images/Spurious_copper/01_spurious_copper_01.jpg (def